# Running the groundstate search for 1D quantum Ising model in qtealeaves

We are interested in the Ising model, defined with the Hamiltonian:

$\hat{H} = - J\sum_{<ij>}\sigma_x^i \sigma_x^j - h \sum_i \sigma_z^i $

We would like to find the groundstate energy and measure local observable $<\sigma_z^i>$ on every site $i$ and correlation matrix $<\sigma_x^i \sigma_x^j>$.

In [ ]:
import numpy as np
import qtealeaves as qtl

## Example 1: 
**Most simple setup** - one system size, one bond dimension, one external field value

In [ ]:
# number of particles
size = 8
# external field value
external_field = 0.5

### Model and operators needed for the model 
In this case we want to run simulations for quantum Ising model in 1D with open boundary conditions.
There is the build-in function for getting this model, but we can also build it from scratch

**Option #1:** building a model from scratch

In [ ]:
from qtealeaves import modeling

# params will be the dictionary with the information of interaction strength, field strength, system size, etc...

# Hamiltonian will depend on the value of external field "g"
model_name = lambda params: "QIsing_g%2.4f" % (params["g"])

# Define a general quantum model - 1-dimensional, of size "L", with a given name
model = modeling.QuantumModel(dim=1, lvals="L", name=model_name)

# which operators we need for defining a model - we need spin-1/2 operators
# each operator is labeled with a string - we need "sz", "sx"
# spin-1/2 operators are already defined within the library, but in principle
# one can define an arbitrary operator
my_ops = qtl.operators.TNSpin12Operators()

# Add the Hamiltonian terms
model += modeling.LocalTerm(operator="sz", strength="g", prefactor=-1)
model += modeling.TwoBodyTerm1D(
    operators=["sx", "sx"], shift=1, strength="J", prefactor=-1, has_obc=True
)

**Option #2:** use a built-in function

In [ ]:
model, my_ops = qtl.models.get_quantum_ising_1d()

### Observables

observables = quantities which we want to measure in a simulation

In this example, we measure:
- energy
- local observable $<\sigma_z>$ at every site
- correlations $<\sigma_x \sigma_x>$ over all sites

In [ ]:
# first define a general TNObservables class
my_obs = qtl.observables.TNObservables()

# then we add in it the observables that we want - energy is measured by default
my_obs += qtl.observables.TNObsLocal(name = '<z>',
                                     operator = 'sz')
my_obs += qtl.observables.TNObsCorr(name = '<xx>',
                                    operators = ['sx', 'sx'])


### Convergence parameters
For groundstate search we need:

- **Maximal bond dimension**: maximal bond dimension that can be reached in a tensor network = in every SVD decomposition we keep maximally "max_bond_dim" values

- **Cut ratio**: cut ration after which the singular values are discarded = when rescaling all the singular values with the largest singular value, we discard of them which are smaller than "cut_ratio"

- **Number of iterations** : number of sweeps in the DMRG groundstate search

In [ ]:
from qtealeaves.convergence_parameters import TNConvergenceParameters

max_bond_dim = 10
cut_ratio = 1e-6
max_iter = 3

# we put it all into the TNConvergenceParameters object
conv_params = TNConvergenceParameters(max_bond_dimension = max_bond_dim,
                                      cut_ratio = cut_ratio,
                                      max_iter = max_iter,
                                      data_type='D', # double precision real
                                      device='cpu', # we are running on CPUs
                                      )

### Input/output folders

- in input folder the library writes all the neccessary input files for the simulation
- in output folder the results will be stored

In [ ]:
input_folder = "folder_in/"
output_folder = "folder_out"

### Simulation

Now we are ready to define the simulation with all this input. In qtealeaves, simulations are defined as the QuantumGreenTeaSimulation class.

In addition, we need to specify the following:
- tensor network type: 5 is for TTN, 6 is for MPS
- tensor backend: 2 means QTeaTensor



In [ ]:
simulation = qtl.QuantumGreenTeaSimulation(model, my_ops, conv_params, my_obs,
                                    tn_type=5,
                                    tensor_backend=2,
                                    folder_name_input=input_folder,
                                    folder_name_output=output_folder,
                                    store_checkpoints=False
    )

### Set the parameters and run the simulation

In [ ]:
# a dictionary which contains parameter values
params = {'L' : size,
        'J' : 1.0,
        'g' : external_field,
            }

simulation.run(params, delete_existing_folder=True)

### Get the results

In [ ]:
results = simulation.get_static_obs(params)

energy = results['energy']
localz = results['<z>']
corrxx = results['<xx>']

print('Energy = ', energy)
print('Local z = ', localz)
print('Corrxx = ', corrxx)

## Example 2 - parametrizing the input

Input can be parametrized, meaning that the actual values of the input parameters do not to be explicitly specified for defining the simulations. The actual values are evaluated only when we run the actual simulation.

One simulation ---> supports multiple input

In [ ]:
# model, operators
model, my_ops = qtl.models.get_quantum_ising_1d()

# observables
my_obs = qtl.observables.TNObservables()

sizes = [4, 8]
external_fields = [0.1, 0.2, 0.5]

max_bond_dim = 10
max_iter = 3

# we put it all into the TNConvergenceParameters object
conv_params = TNConvergenceParameters(max_bond_dimension = 10,
                                      max_iter = max_iter,
                                      data_type='D', # double precision real
                                      device='cpu', # we are running on CPUs
                                      )


### Parametrize the input/output folders

In [ ]:
input_folder = lambda params : 'input_L%02d_g%2.4f'%(
        params['L'],
        params['g'],
    )

output_folder = lambda params : 'output_L%02d_g%2.4f'%(
        params['L'],
        params['g'],
    )

### Set up simulation as before

In [ ]:
#print('my conv = ', conv_params.max_bond_dimension)
simulation = qtl.QuantumGreenTeaSimulation(model, my_ops, conv_params, my_obs,
                                    tn_type=5,
                                    tensor_backend=2,
                                    folder_name_input=input_folder,
                                    folder_name_output=output_folder,
                                    store_checkpoints=False
    )

### Add all the parameter combinations to the list of parameter dictionaries

In [ ]:
import itertools

params = []

# itertools: 1st one iterates slowly
for size, external_field in itertools.product(sizes, external_fields):
    params.append({
            'L' : size,
            'J' : 1.0,
            'g' : external_field,
       })

for elem in params:
    print(elem)

### Run the simulation for each of the parameter combinations

In [ ]:
for elem in params:
    print('params = ', elem)
    simulation.run(elem, delete_existing_folder=True)

    results = simulation.get_static_obs(elem)

    energy = results['energy']

    print('Energy = ', energy)

## Example 3
**Running multiple parameters in parallel on a cluster**

There is a simple way of launching the parallel simulations with different input on a cluster via the single script. The setup is the same as above up to the part when we run the actual simulations.

For running in parallel, go with:

`simulation.run(params, delete_existing_folder=True, nthreads=<num_parallel_simulations>)`,

where `params` is a list of input parameters as in Example 2.

**Remark:**
In this case, you must specify the number of MKL and OMP threads in the sbatch script to be equal to 1, i.e. add the following lines to the sbatch script:

`export MKL_NUM_THREADS=1`

`export OMP_NUM_THREADS=1`

This ensures that the simulations are parallelized over the input, and not parallelizing the numpy operations which can be done by multithreading on multiple CPUs.